# COMPSCI 367 23S2 Assignment 1

- Lecturer: Anna Trofimova
- School of Computer Science, The Univerity of Auckland
- Last update 6th of August at 18:00pm, 2023
$$
% macros
\newcommand{\indep}{\perp \!\!\!\perp}
$$

## Student:

Shinbeom Kwon, bkwo461

### Submission

This interactive notebook contains the instructions to complete assignment 1; You should submit this notebook with the code and answers in one single file in .ipybn format with name assignment1.ipybn. **Write your name and UPI in the cell above** (to edit the markdown text double-click the cell).

There is a maximum file size cap of 5MB, so make sure your submission does not exceed this size. The submitted notebook should contain all your source code and answers. You can add new cells and use markdown text to organise and explain your implementation/answer.

Late submission policy: for each day past the deadline, there's a 20% deduction from the maximum possible score. By the fifth day post-deadline, the highest score attainable is 0. Example: if your submission received an 80% grade, but it was turned in 2 days late, then your final score is capped at 60%.

Submit this notebook using CANVAS upload function.


**The submission deadline is 18th of August at 11.59pm, 2023.**

### Plagiarism
This is an individual assignment. Remember that **all** work submitted for this assignment must be your own **individual** work and no code sharing or copying is allowed. You may use code from the Internet only with suitable attribution of the source in your program. Using code generation tools does not excuse you from the responsibility of ensuring the originality and authenticity of your work, and you should refrain from submitting code generated by AI as your own without proper attribution. **Do not use public code repositories as your code might be copied.** Keep in mind that sharing parts of assignment solutions is a form of plagiarism. All submitted assignments will be run through plagiarism detection software to detect similarities to other submissions. It is your responsibility to familiarise yourself with the UoA policy on academic integrity and plagiarism. 

### Description

This assignment consist of 5 parts with the following distribution of the mark:
- part 1 is worth 5% of the total mark
- part 2 is worth 25% of the total mark
- part 3 is worth 35% of the total mark
- part 4 is worth 25% of the total mark
- part 5 is worth 15% of the total mark

For solving the problems in this assignment, please utilize the **aipython** package. This package offers a variety of pre-implemented classes and usage examples. To download the most recent version of the package, follow this link: https://artint.info/AIPython/aipython.zip

Place this notebook in a folder next to aipyton folder (not inside it).

When working with a Jupyter notebook, you can edit the \*.ipynb files either in the Jupyter interface (in your browser) or with your favorite editor (e.g. PyCharm). Whenever you save a \*.ipynb file, the notebook will reload its content directly. Whenever you execute a cell, the output is displayed below that cell and remains saved within the notebook - do not clear the output.

The libraries that you can use (and need) in this assignment are  the following (run the cell below to import the libraries):

In [1]:
import sys
sys.path.insert(0, "./aipython")

import time
import numpy as np
from aipython.searchGeneric import *
from aipython.searchProblem import *
from aipython.cspProblem import *
import random
import copy
import math

If you want to use a library which is not in this list, you need to get an explicit permission from the lecturer (anna.trofimova@auckland.ac.nz)

## Part 1 - State Space Problem & Search [5%]

This part of the assignment is based on a popular puzzle, Game of Fifteen, also known as 15-puzzle. The game consists of 15 numbered tiles on a four by four grid where one tile is missing. The objective is to rearrange the tiles by moving the empty space to order them from 1 to 15. The goal state of the puzzle is defined as a multidimensional array of digits, where 0 represents the missing tile, as follow:

In [2]:
goal = [[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]]

#### Task 1.1 Creating a FifteenPuzzleGame class

Edit the code below to implement a search problem class representing the Game of Fifteen. The class should be an extension of Search_problem class and the heuristic function should be implemented as a Manhattan distance.

In [3]:
class GameFifteenProblem(Search_problem):
    """A class that represents 15 Puzzle Game problem (with Manhattan heuristic)
        start - 2D array of numbers representing the initial state of the game
        goal - 2D array of numbers representing the goal stay of the game
    """

    def __init__(self, start, goal):  
        self.start = start
        self.goal = goal

    def start_node(self):
        return self.start

    def is_goal(self, node):
        return node == self.goal
    
    def heuristic(self, node):
        total_dis = 0
        for i in range(len(node)):
            for j in range(len(node[i])):
                if node[i][j] != 0:
                    target = node[i][j]
                    for k in range(len(self.goal)):
                        for p in range(len(self.goal[k])):
                            if self.goal[k][p] == target:
                                total_dis += (abs(i-k) + abs(j-p))
        return total_dis
                    
    def neighbors(self, node):
        neighbors = []
        zero_x, zero_y = None, None
        for i in range(len(node)):
            for j in range(len(node[i])):
                if node[i][j] == 0:
                    zero_x, zero_y = i, j
                    break
            if zero_x is not None:
                break
        
        if zero_x > 0:
            new_node = [row[:] for row in node]
            new_node[zero_x][zero_y], new_node[zero_x-1][zero_y] = new_node[zero_x-1][zero_y], new_node[zero_x][zero_y]
            neighbors.append(Arc(node, new_node, 1))
        
        if zero_x < len(node) - 1:
            new_node = [row[:] for row in node]
            new_node[zero_x][zero_y], new_node[zero_x+1][zero_y] = new_node[zero_x+1][zero_y], new_node[zero_x][zero_y]
            neighbors.append(Arc(node, new_node, 1))
        
        if zero_y > 0:
            new_node = [row[:] for row in node]
            new_node[zero_x][zero_y], new_node[zero_x][zero_y-1] = new_node[zero_x][zero_y-1], new_node[zero_x][zero_y]
            neighbors.append(Arc(node, new_node, 1))
        
        if zero_y < len(node[0]) - 1:
            new_node = [row[:] for row in node]
            new_node[zero_x][zero_y], new_node[zero_x][zero_y+1] = new_node[zero_x][zero_y+1], new_node[zero_x][zero_y]
            neighbors.append(Arc(node, new_node, 1))
        
        return neighbors

        

To validate the correctness of the problem class you cab use the A* searcher algorithm (from searchGeneric.py) to find a solution.

*The cost of the solution should be 7.*

In [4]:
start_test = [[1, 2, 3, 4],
             [9, 5, 6, 7],
             [10, 0, 11, 8],
             [13, 14, 15, 12]]

puzzle = GameFifteenProblem(start_test, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

8 paths have been expanded and 17 paths remain in the frontier
Cost:  7


## Part 2 - Performance Evaluation [25%]


#### Task 2.1 Implement IDA*
Edit the code below to implement a class representing Iterative Deepening A* Search. The class should be an extension of Searcher class and the search should terminate when the solution is found or when the limit - the boundary for the evaluation value,  cannot be updated any further.

In [5]:
class IterativeDeepeningAStarSearcher(Searcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):
        
        self.threshold = 100000
        self.thre = []
        super().__init__(problem)
        

    def add_to_frontier(self, path):
        if path.cost + self.problem.heuristic(path.end()) <= self.threshold:
            super().add_to_frontier(path)
        else:
            if path.cost + self.problem.heuristic(path.end()) not in self.thre:
                self.thre.append(path.cost + self.problem.heuristic(path.end()))

    @visualize
    def search(self):
        
        self.thre.append(self.problem.heuristic(self.problem.start_node()))       
        while True:
            self.thre.sort(reverse = True)
            self.threshold = self.thre.pop()    #thre = []
        
            while not self.empty_frontier():
                path = self.frontier.pop()
                self.display(2, "Expanding:",path,"(cost:",path.cost,")")
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):    # solution found
                    self.display(1, self.num_expanded, "paths have been expanded and",
                                len(self.frontier), "paths remain in the frontier")
                    self.solution = path   # store the solution found
                    return path
                else:
                    neighs = self.problem.neighbors(path.end())
                    self.display(3,"Neighbors are", neighs)
                    for arc in reversed(list(neighs)):
                        self.add_to_frontier(Path(path,arc))
                    self.display(3,"Frontier:",self.frontier)
            self.display(1,"No (more) solutions. Total of",
                        self.num_expanded,"paths expanded.")
            self.threshold = 100000
            self.initialize_frontier()
            self.add_to_frontier(Path(self.problem.start_node()))   #back to start node so that it could ref

#### Task 2.2 Record Search Performance

In this task you will compare the properties of different search algorithms. You will need to run Uniform Cost Search, Greedy Search, A* Search and Iterative Deepening A* Search on each of the start states below:

In [6]:
start6 = [[1, 2, 7, 3],
          [5, 6, 11, 4],
          [9, 10, 0, 8],
          [13, 14, 15, 12]]

start12 = [[1, 3, 4, 0],
           [5, 2, 6, 7],
           [9, 10, 15, 8],
           [13, 14, 12, 11]]

start25 = [[10, 11, 6, 3],
           [2, 1, 0, 4],
           [5, 8, 15, 7],
           [9, 13, 14, 12]]

start33 = [[2, 7, 0, 4],
           [6, 3, 11, 12],
           [1, 5, 15, 14],
           [9, 10, 8, 13]]

start41 = [[7, 11, 12, 4],
           [2, 5, 3, 14],
           [10, 0, 8, 1],
           [6, 9, 13, 15]]

Use the implementation of A* Search from aipython package and the implementation of Uniform-Cost Search and Greedy Search from the cell below.

In [7]:
class UniformCostSearcher(AStarSearcher):
    """returns a greedy searcher for a problem.
    Paths can be found by repeatedly calling search().
    """

    def __init__(self, problem):
        super().__init__(problem)

    def add_to_frontier(self, path):
        """add path to the frontier with the appropriate cost"""
        value = path.cost
        self.frontier.add(path, value)
        return

class GreedyBestSearcher(AStarSearcher):
    """returns a greedy searcher for a problem.
    Paths can be found by repeatedly calling search().
    """

    def __init__(self, problem):
        super().__init__(problem)

    def add_to_frontier(self, path):
        """add path to the frontier with the appropriate cost"""
        value = self.problem.heuristic(path.end())
        self.frontier.add(path, value)
        return

In [8]:
puzzle = GameFifteenProblem(start_test, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

8 paths have been expanded and 17 paths remain in the frontier
Cost:  7


In each case, record in the table below the number of nodes generated during the search. If the number of generated states exceeds 800000 items, just write “Mem” in your table. If the code runs for 1 minute without producing output, terminate the process and write “Time” in your table.

*Tip: To edit the table double click the cell below.*

In [9]:
puzzle = GameFifteenProblem(start6, goal)
searcher = IterativeDeepeningAStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

7 paths have been expanded and 0 paths remain in the frontier
Cost:  6


| Algorithm | 6       | 12      | 25      | 33      | 41      |
|-----------|---------|---------|---------|---------|---------|
|UCS        | 4017    | 50807   | Time    | Time    | Time    |
|GBS        | 13      | 23      | 63      | Time    | Time    |
|A*         | 13      | 23      | 63      | 39447   | Time    |
|IDA\*S     | 0       | 0       | 7       | 17      | 29      |

#### Task 2.3 Performance Evaluation

In the cell below briefly evaluate the time and space performance of each algorithm using the data from the table. Identify the most efficient algorithm and mention any results that differ from your expectations. Limit your answer to 100 words.

*Tip: To edit the answer double click the cell below.*

As can be seen from the result table, the most optimized algorithm is the IDA* search algorithm. By setting thresholds and continuing to update them, it was the fastest and least memory was required because the A* search algorithm was used, significantly reducing the probability of going down the path to failure. 
It can be seen that UCS consumes the most memory and time, and GBS also consumes a lot of time to find the best path. These are all results derived because only cost or heuristic values are tracked.

**Answer:** Insert your answer here.

## Part 3 - Game State Generation [35%]


#### Task 3.1 Implement Game Generator

Edit the code in the cell below to implement a class 'GameGenerator' which generates a Game of Fifteen with a specified cost of the optimal solution. The method 'generate' should return a 2D array representing the game state with the specified path cost. Please limit the scope of your implementation to the approaches/algorithms covered in the course.

Your implementation will be evaluated based on the correctness of the output, the runtime, and the extent of code reuse.

In [10]:
from copy import deepcopy


class GameGenerator:
    
    def __init__(self, path_cost):
        self.path_cost = path_cost
        self.goal = goal
        self.result = self.goal.copy()
        self.path = Path(self.result)
    
    def generate(self):
        
        while True:
            steps = []
            heuris = []
            problem = GameFifteenProblem(self.result, self.goal)
            sol = IterativeDeepeningAStarSearcher(problem).search()
            if sol.cost == self.path_cost:
                return self.result
            neighs = problem.neighbors(self.result)
            
            for arc in reversed(list(neighs)):
                itsheuri = problem.heuristic(Path(self.path, arc).end())
                heuris.append(itsheuri)
                steps.append(Path(self.path, arc))
            want = heuris.index(max(heuris))
            self.path = steps[want]
            self.result = self.path.end()
    
    def get_result(self):

        game = GameGenerator(self.path_cost)
        myGame = game.generate()
        return myGame
        
        
            

In [11]:
GameGenerator(40).get_result()

1 paths have been expanded and 0 paths remain in the frontier
2 paths have been expanded and 0 paths remain in the frontier
3 paths have been expanded and 0 paths remain in the frontier
4 paths have been expanded and 0 paths remain in the frontier
5 paths have been expanded and 0 paths remain in the frontier
6 paths have been expanded and 0 paths remain in the frontier
7 paths have been expanded and 0 paths remain in the frontier
8 paths have been expanded and 0 paths remain in the frontier
9 paths have been expanded and 0 paths remain in the frontier
10 paths have been expanded and 0 paths remain in the frontier
11 paths have been expanded and 0 paths remain in the frontier
12 paths have been expanded and 0 paths remain in the frontier
13 paths have been expanded and 0 paths remain in the frontier
20 paths have been expanded and 0 paths remain in the frontier
21 paths have been expanded and 1 paths remain in the frontier
22 paths have been expanded and 2 paths remain in the frontier
2

100 paths have been expanded and 5 paths remain in the frontier
101 paths have been expanded and 5 paths remain in the frontier
133 paths have been expanded and 5 paths remain in the frontier
134 paths have been expanded and 6 paths remain in the frontier
135 paths have been expanded and 6 paths remain in the frontier
136 paths have been expanded and 6 paths remain in the frontier
137 paths have been expanded and 6 paths remain in the frontier
153 paths have been expanded and 6 paths remain in the frontier
169 paths have been expanded and 7 paths remain in the frontier
170 paths have been expanded and 8 paths remain in the frontier
207 paths have been expanded and 8 paths remain in the frontier
208 paths have been expanded and 8 paths remain in the frontier
209 paths have been expanded and 8 paths remain in the frontier
322 paths have been expanded and 9 paths remain in the frontier


[[8, 5, 1, 15], [4, 0, 2, 3], [11, 7, 6, 13], [10, 12, 9, 14]]

#### Task 3.2 Game Generator Intuition

In the cell below describe the algorithm or method you employed to develop the GameGenerator class. Detail how you ensured the correctness of the output and optimized the generation speed. Limit your answer to 100 words.

*Tip: To edit the answer double click the cell below.*

The core of the problem was to create an initial state, the start state, using the given path cost and goal state. All search algorithms equally explore the optimal path from the initial state to the target state in heuristic or cost, so I thought the opposite. We considered a given goal state as a start state and used the IDA* search algorithm to write a code that searches for a given cost. Correctness will be guaranteed because IDA* was used. In addition, the path was constructed with nodes with the largest heuristic value in order to search for the optimal value.

## Part 4 - Graph Search [25%]

The cell below specifies a state space problem as a graph, where the start state is the node S and the goal state is the node G. The arc cost and the heuristic values for each state are also provided.

In [12]:
problem = Search_problem_from_explicit_graph(
    {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S'},
    [Arc('A', 'H', 1), Arc('A', 'K', 1), Arc('A', 'R', 7), Arc('B', 'O', 7), Arc('B', 'R', 2), Arc('B', 'S', 3),
     Arc('C', 'E', 1), Arc('C', 'N', 1), Arc('C', 'R', 6), Arc('D', 'B', 5), Arc('D', 'L', 6), Arc('E', 'B', 1),
     Arc('E', 'K', 1), Arc('E', 'M', 6), Arc('F', 'G', 5), Arc('F', 'J', 3), Arc('G', 'C', 2), Arc('G', 'F', 2),
     Arc('G', 'J', 2), Arc('H', 'D', 5), Arc('H', 'E', 5), Arc('H', 'J', 7), Arc('H', 'M', 6), Arc('J', 'C', 1),
     Arc('J', 'D', 2), Arc('J', 'G', 7), Arc('J', 'H', 3), Arc('J', 'Q', 2), Arc('J', 'S', 5), Arc('K', 'C', 6),
     Arc('K', 'L', 5), Arc('K', 'N', 5), Arc('K', 'S', 2), Arc('M', 'E', 6), Arc('M', 'O', 7), Arc('N', 'C', 1),
     Arc('N', 'E', 1), Arc('N', 'M', 5), Arc('O', 'C', 2), Arc('O', 'R', 1), Arc('P', 'B', 4), Arc('P', 'E', 5),
     Arc('P', 'F', 1), Arc('P', 'H', 7), Arc('P', 'J', 4), Arc('P', 'K', 2), Arc('P', 'M', 2), Arc('P', 'R', 3),
     Arc('P', 'S', 1), Arc('Q', 'J', 7), Arc('Q', 'K', 1), Arc('Q', 'N', 6), Arc('Q', 'P', 2), Arc('R', 'E', 2),
     Arc('R', 'O', 1), Arc('R', 'Q', 6), Arc('R', 'S', 7), Arc('S', 'E', 1), Arc('S', 'M', 2), Arc('S', 'N', 3)],
    start='S',
    goals={'G'},
    hmap={'A': 15, 'B': 19, 'C': 18, 'D': 21, 'E': 20, 'F': 10, 'G': 0, 'H': 10, 'J': 7, 'K': 17, 'L': 19, 'M': 22,
          'N': 18, 'O': 15, 'P': 12, 'Q': 12, 'R': 18, 'S': 18})

#### Task 4.1 Graph Search

In the cell below write the code that finds an optimal solution to the problem above. Please limit the scope of your implementation to the approaches/algorithms covered in the course. Print the solution in the format: 
S --> A --> B --> C --> G.

Your implementation will be evaluated based on the correctness of the output, the runtime, and the extent of code reuse.

In [13]:
# insert you code here
from aipython.searchBranchAndBound import *

class FindOptimal(AStarSearcher):
    def __init__(self, problem):    
        self.problem = problem
        self.low_bound = UniformCostSearcher(self.problem).search().cost
        self.result_path = None
        self.result_cost = self.low_bound
        self.start_node = problem.start_node()
        self.goal_nodes = problem.goals
        super().__init__(problem)

    def is_consistency(self, arc, from_node, to_node):
        if self.problem.heuristic(from_node) <= self.problem.heuristic(to_node) + arc.cost:
            return True
        return False
    

    def findSolution1(self):
        self.add_to_frontier(Path(self.start_node))
        
        while not self.empty_frontier():
            path = self.frontier.pop()
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):    # solution found
                self.display(1, self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                self.solution = path   # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    if self.is_consistency(arc, arc.from_node, arc.to_node):
                        self.add_to_frontier(Path(path, arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")
        
    def findSolution2(self):
        self.add_to_frontier(Path(self.start_node))
        result = IterativeDeepeningAStarSearcher(self.problem).search() 
        return result
    
    def findSolution3(self):
        while True:
            mysolution = DF_branch_and_bound(self.problem, self.result_cost)
            sol = mysolution.search()
            if sol == None:
                self.result_cost += 1
                continue
            result = sol
            while True:
                new_sol = mysolution.search()
                if new_sol == result:
                    return result



        

In [14]:
FindOptimal(problem).findSolution1()
print()
FindOptimal(problem).findSolution2()
print()
FindOptimal(problem).findSolution3()

7819 paths have been expanded and 16432 paths remain in the frontier
22 paths have been expanded and 46 paths remain in the frontier



7819 paths have been expanded and 16432 paths remain in the frontier
No (more) solutions. Total of 1 paths expanded.
No (more) solutions. Total of 6 paths expanded.
No (more) solutions. Total of 16 paths expanded.
No (more) solutions. Total of 28 paths expanded.
40 paths have been expanded and 5 paths remain in the frontier

7819 paths have been expanded and 16432 paths remain in the frontier
Number of paths expanded: 0 (no solution found)
Number of paths expanded: 1 (no solution found)
Number of paths expanded: 1 (no solution found)
Number of paths expanded: 1 (no solution found)
Number of paths expanded: 5 (no solution found)
Number of paths expanded: 10 (no solution found)
Number of paths expanded: 12 (no solution found)
Number of paths expanded: 18 (optimal solution found)
Number of paths expanded: 12 (optimal solution found)


S --> E --> B --> R --> Q --> J --> G

#### Task 4.2 Graph Search Intuition

In the cell below describe the algorithm or method you employed to find an optimal solution in the previous task. Explain your choice. Limit your answer to 100 words.

*Tip: To edit the answer double click the cell below.*

I made codes in three ways to find the optimal path from the starting node to the goal node of a given problem. FindSolution1 generated a function to determine whether heuristic was consistent in order to obtain the path in an optimal way.  Since hmap is given, if we only check consistency, the heuristic is suitable for finding the optimal path. FindSolution2 is obtained using only the existing IDA* algorithm. FindSolution3 is an optimal path search function using recursion using the DF_branch_and_bound function.

## Part 5 - Queens Conflicts [15%]

The goal of 8-Queens problem is to place 8 queens on an 8x8 chessboard so that no two queens threaten each other. This means that no two queens can be in the same row, column, or diagonal.

Consider the following state of the problem where 0 represents empty square and 1 represents a square with a queen:


In [15]:
board_start = [[0, 1, 0, 0, 0, 0, 0, 0],
               [0, 1, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0, 0, 1],
               [0, 0, 1, 0, 0, 0, 0, 0],
               [0, 0, 1, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 1, 0, 0],
               [0, 0, 0, 0, 1, 0, 0, 0]]

#### Task 5.1 Count conflicts

In the cell below implement the function conflicts which take a state of the chessboard as an argument and returns the number of conflicts for it.

In [16]:
# def conflicts(board):
#     result = 0
#     #row search
#     for i in range(8):
#         count = 0
#         for j in range(8):
#             if board[i][j] == 1:
#                 count += 1
#         if count > 1:
#             for num in range(1, count):
#                 result += num
#     #column search
#     for i in range(8):
#         count = 0
#         for j in range(8):
#             if board[j][i] == 1:
#                 count += 1
#         if count > 1:
#             for n in range(1, count):
#                 result += n
#     #row search
#     for i in range(8):
#         count = 0
#         plus = 0
#         for j in range(8-i):
#             if board[i+plus][j] == 1:
#                 count += 1
#             plus += 1
#         if count > 1:
#             for n in range(1, count):
#                 result += n

#     #upward diagonal search
#     for i in range(6):
#         count = 0
#         add = 0
#         for j in range(7-i):
#             if board[j+add][j+add+1] != None and board[j+add][j+add+1] == 1:
#                 count += 1
#             add += 1
#         if count > 1:
#             for n in range(1, count):
#                 result += n
    
#     
#     return result


In [17]:
def conflicts(board):
    board = np.array(board)                 
    queen_positions = np.argwhere(board == 1)
    count = 0
    for i in range(len(queen_positions)):
        for j in range(i+1,len(queen_positions)):
            
            if queen_positions[i][1] == queen_positions[j][1]:
                count += 1
                
            if queen_positions[i][0] == queen_positions[j][0]:
                count += 1
                
            if (abs(queen_positions[i][0]-queen_positions[j][0]) == abs(queen_positions[i][1]-queen_positions[j][1])):
                count += 1
                
    
    return count

In [18]:
conflicts(board_start)

5

#### Task 5.2 Local Search

In the cell below develop the function queens_with_conflicts which implements a local search approach to finding a state of 8-queens that has specified number of conflicts. The function should take a 2D array representing the initial board and the number of conflicts as input, and then return a 2D array representing the solution.

Your implementation will be evaluated based on the correctness of the output, the runtime, and the extent of code reuse.

In [19]:
import copy
import random

def queens_with_conflicts(board, c):
    n = len(board)
    
    def get_queen_positions(current_board):
        positions = [0] * n
        for col in range(n):
            for row in range(n):
                if current_board[row][col] == 1:
                    positions[col] = row
                    break
        return positions
    
    def move_queen(current_board, from_row, to_row, col):
        current_board[from_row][col] = 0
        current_board[to_row][col] = 1
        return current_board
    
    def find_candidate_moves(current_board, queen_positions):
        moves = []
        for col, from_row in enumerate(queen_positions):
            for to_row in range(n):
                if to_row == from_row:
                    continue

                myboard = copy.deepcopy(current_board)
                myboard = move_queen(myboard, from_row, to_row, col)

                new_conflicts = conflicts(myboard)
                moves.append((myboard, new_conflicts))
        return moves
    
    def stochastic_select(candidates):
        return random.choice(candidates)

    queen_positions = get_queen_positions(board)
    
    while conflicts(board) > c:
        candidates = find_candidate_moves(board, queen_positions)
        (new_board, new_conflicts) = stochastic_select(candidates)
        board = new_board
        queen_positions = get_queen_positions(board)

    return board


In [20]:
queens_with_conflicts(board_start, 3)

[[0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 0, 0]]

#### Task 5.3 Queens with Conflicts Intuition

In the cell below explain the choice of your local search algorithm and intuition behind the function queens_with_conflicts.Limit your answer to 100 words.

*Tip: To edit the answer double click the cell below.*

Among local searches, stochastic local search was used. The code was configured to repeat the loop until the number of conflicts equals c when the queen moves randomly. In this case, the location of the queen is a one-dimensional list rather than a 2d array list. This is to reduce the runtime to find the location of the queen. Queen's domain was placed in the candidates list and the code was constructed so that it could be randomly selected from the stochastic_select function.